In [ ]:
#
# Setup constants
#

import subprocess

MODEL_NAME = "giia"
MODEL_VERSION = "0.3.3"
# DATASET = "datasets/SandP_1995_2019_monthly.csv"
DATASET = "datasets/SandP_1995_2020_daily.csv"
SM_ROLE ='arn:aws:iam::941048668662:role/service-role/AmazonSageMaker-ExecutionRole-20191206T145896'

AWS_INSTANCE = 'ml.m5.large'
LOCAL_INSTANCE = 'local'
try:
    if subprocess.call('nvidia-smi') == 0:
        LOCAL_INSTANCE = 'local_gpu'
except:
    print("The nvidia-smi binary was not found and thus GPU computation is not supported. Using the default CPU "
          "computation")

# Change this to your desired instance type
INSTANCE_TYPE = LOCAL_INSTANCE
IS_LOCAL = LOCAL_INSTANCE==INSTANCE_TYPE

In [ ]:
#
# Initialization
#

import os
import sys
from pathlib import Path

# Set notebook's src module path. Note that you may have to update your IDE's project settings to do the same for the
#  local library imports to work the same
module_path = Path().resolve()
sys.path.append(module_path)

# Keep paths consistent throughout notebook
os.chdir(module_path)

# Autoreload imports at the beginning of cell execution.
#  https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

from utils.logging import LoggerUtil
from utils.utils import Utils

LOGGER = LoggerUtil(f"{MODEL_NAME}-{MODEL_VERSION}")
UTILS = Utils(LOGGER)

LOGGER.log("Current working directory [{}]".format(os.getcwd()))
UTILS.describe_env()

In [ ]:
#
# Parse dataset
#

from data_processing.parse import Parse

PARSE = Parse(LOGGER)

# Creates train and test dataset CSVs
train_dataset, test_dataset = PARSE.split_train_test_dataset(DATASET)

In [ ]:
#
# Setup local/aws environment. If aws, upload the datasets to S3
#

from data_processing.upload import Upload

UPLOAD = Upload(LOGGER)

train_dataset_path, test_dataset_path = UPLOAD.upload_to_sagemaker_s3_bucket(
    f"{MODEL_NAME}-{MODEL_VERSION}", train_dataset, test_dataset)

In [ ]:
#
# Configure sagemaker and estimator
#

from ml.train import Train

TRAIN = Train(LOGGER)

estimator = TRAIN.create_model(SM_ROLE, INSTANCE_TYPE)
TRAIN.fit_model(estimator, train_dataset_path, test_dataset_path)

In [ ]:
#
# NOTE: FURTHER CELLS ARE COMPATIBLE WITH AWS SAGEMAKER ONLY, LOCAL MODE WILL NOT WORK
# Hyperparameter tune the model
#

from ml.tune import Tune

TUNE = Tune(LOGGER)

tuner = TUNE.create_tuner(estimator)
tuning_job_name = TUNE.fit_tuner(tuner, train_dataset_path, test_dataset_path)

In [ ]:
#
# Get updates for Hyperparameter tune job. Ensure this is completed before going to the next cell
#

TUNE.get_tune_job_update()

In [ ]:
#
# Evaluate the metrics of the tune job
#

TUNE.report_job_analytics()
